# Addestramento modelli

In [ ]:
import warnings
import numpy as np
import pandas as pd
import os
import sys

In [ ]:
warnings.filterwarnings("ignore")
sys.path.insert(0, "..")

In [ ]:
import utility

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder

In [ ]:
import tensorflow as tf
from tensorflow import Tensor 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Concatenate, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Attention, AdditiveAttention
from tensorflow.keras.layers import Activation, BatchNormalization, Add, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import L1, L2, L1L2

In [ ]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers.merge import concatenate

In [ ]:
K.set_epsilon(1)

In [ ]:
model_trained_base_path = "TrainingDirectory"
dataset_path = os.path.join("..", "Preprocessing", "Dataset", "dataset_training.csv")

### Import dei dati

In [ ]:
df = utility.import_dataset(dataset_path, "2021", "2023")
display(df)

### OneHot encoding e normalizzazione

In [ ]:
df, scaler = utility.processing_dataset(df)

### Splitting in training, validation e test

In [ ]:
df_train, df_validation, df_test = utility.split_dataset(df, "2021-03", "2022-04")

### Model 1 - PAPER0 - LSTMwithoutOtherFeature

In [ ]:
def create_model_1():

    dropout_rate = 0.3
    regularizer = None
    learning_rate = 0.0001
    
    batch_size = 256

    time_steps = 24 * 7
    dim_other_feature = 24 + 7 + 12 + 1
    
    model = Sequential()
    model.add(Input(shape=(time_steps, 1), name="CaricoElettrico"))
    model.add(LSTM(units=128, return_sequences=True, activity_regularizer=regularizer, name="LSTM_1"))
    model.add(LSTM(units=128, activity_regularizer=regularizer, name ="LSTM_2"))
    model.add(Dropout(rate=dropout_rate, name="Dropout"))
    model.add(Dense(units=1, activation="linear", name="Predizione"))
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model, time_steps, batch_size

### Model 2 - PAPER12 - LSTMwithOtherFeature

In [ ]:
def create_model_2():
    
    dropout_rate = 0.4
    regularizer = L2(0.1)
    learning_rate = 0.0001
    
    batch_size = 128

    time_steps = 24 * 3
    dim_other_feature = 24 + 7 + 12 + 1
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1), name="CaricoElettrico")
    lstm_layer = LSTM(64, return_sequences=False, activity_regularizer=regularizer, name="LSTM")(input_load)
    dropout1 = Dropout(rate=dropout_rate, name="Dropout_1")(lstm_layer)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ), name="FeatureAggiuntive")
    dense1 = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC_1")(input_other_feature)
    dropout2 = Dropout(rate=dropout_rate, name="Dropout_2")(dense1)

    # Concatenate PIPLINES
    concatenated_layer = concatenate([dropout1, dropout2], axis=1)
    dense2 = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC_2")(concatenated_layer)
    dropout3 = Dropout(rate=dropout_rate, name="Dropout_3")(dense2)
    output_layer = Dense(1, activation="linear", activity_regularizer=regularizer, name="Predizione")(dropout3)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model, time_steps, batch_size

### Model 3 - PAPER13 - Conv1D + LSTM

In [ ]:
def create_model_3():
    
    dropout_rate = 0.5
    regularizer = L2(0.001)
    learning_rate = 0.0001
    
    batch_size = 64

    time_steps = 24 * 3
    dim_other_feature = 24 + 7 + 12 + 1
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1), name="CaricoElettrico")

    # Pipeline 1
    branch1_1 = Conv1D(filters=64, kernel_size=3, activation="relu", activity_regularizer=regularizer, name="Conv_1")(input_load)
    branch1_2 = MaxPooling1D(pool_size=2, strides=2, name="MaxPool_1")(branch1_1)

    # Pipeline 2
    branch2_1 = Conv1D(filters=64, kernel_size=5, activation="relu", activity_regularizer=regularizer, name="Conv_2")(input_load)
    branch2_2 = MaxPooling1D(pool_size=2, strides=2, name="MaxPool_2")(branch2_1)

    # Pipeline 3
    branch3_1 = Conv1D(filters=64, kernel_size=7, activation="relu", activity_regularizer=regularizer, name="Conv_3")(input_load)
    branch3_2 = MaxPooling1D(pool_size=2, strides=2, name="MaxPool_3")(branch3_1)

    # Concatenate pipelines LOAD
    concatenated_layer1 = concatenate([branch1_2,branch2_2,branch3_2], axis=1)
    lstm_layer = LSTM(64, return_sequences=True, activity_regularizer=regularizer, name="LSTM")(concatenated_layer1)
    global_average_pooling_layer = GlobalAveragePooling1D(name="GlobalAveragePooling")(lstm_layer)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ), name="FeatureAggiuntive")
    dense1 = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC_1")(input_other_feature)
    dropout1 = Dropout(rate=dropout_rate, name="Dropout_1")(dense1)
    flatten1 = Flatten()(dropout1)

    # Concatenate PIPLINES
    concatenated_layer2 = concatenate([global_average_pooling_layer,flatten1], axis=1)
    dense2 = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC_2")(concatenated_layer2)
    dropout2 = Dropout(rate=dropout_rate, name="Dropout_2")(dense2)
    dense3 = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC_3")(dropout2)
    dropout3 = Dropout(rate=dropout_rate, name="Dropout_3")(dense3)
    output_layer = Dense(1, activation="linear", activity_regularizer=regularizer, name="Predizione")(dropout3)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model, time_steps, batch_size

### Model 4 - PAPER18 - Wavenet

In [ ]:
def create_model_4():
       
    dropout_rate = 0.3
    regularizer = None
    learning_rate = 0.0001
    
    batch_size = 128

    time_steps = 24 * 5
    dim_other_feature = 24 + 7 + 12 + 1
    
    # Pipeline LOAD
    input_load = Input(shape=(time_steps, 1), name="CaricoElettrico")
    x = input_load
    for rate in (1, 2, 4, 8, 16, 32):
        x = Conv1D(filters=32, kernel_size=2, strides=1, padding="causal", activation="relu", dilation_rate=rate,
                  activity_regularizer=regularizer, name="Conv_"+str(rate))(x)
    flatten = Flatten()(x)

    # Pipeline OTHER FEATURE
    input_other_feature = Input(shape=(dim_other_feature, ), name="FeaturAggiuntive")

    # Concatenate PIPLINES
    concatenated_layer = concatenate([flatten, input_other_feature], axis=1)
    dropout1 = Dropout(rate=dropout_rate, name="Dropout_1")(concatenated_layer)
    dense = Dense(64, activation="relu", activity_regularizer=regularizer, name="FC")(dropout1)
    dropout2 = Dropout(rate=dropout_rate, name="Dropout_2")(dense)
    output_layer = Dense(1, activation="linear", activity_regularizer=regularizer, name="Predizione")(dropout2)
    model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])
    
    # Compiling
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
    
    return model, time_steps, batch_size

### Training model

In [ ]:
base_path = os.path.join(model_trained_base_path, "model2")
os.mkdir(base_path)
model_path = os.path.join(base_path, "weights-model.h5")
history_path = os.path.join(base_path, "history-model.json")

In [ ]:
epochs = 200
patience = 5

In [ ]:
model, time_steps, batch_size = create_model_4()

In [ ]:
model.summary()

In [ ]:
train = utility.windowed_dataset(df_train, time_steps)
validation = utility.windowed_dataset(df_validation, time_steps)
test = utility.windowed_dataset(df_test, time_steps)

In [ ]:
early_stopping = EarlyStopping(monitor="val_mape", patience=patience)

history = model.fit(
    utility.get_x(train, "CaricoTotale"), utility.get_y(train, "CaricoTotale"), 
    epochs=epochs, 
    verbose=1, 
    callbacks=[early_stopping],
    batch_size=batch_size, 
    validation_data=(utility.get_x(validation, "CaricoTotale"), utility.get_y(validation, "CaricoTotale")), 
    shuffle=False
)

history = history.history
utility.save_history(history, history_path)
model.save(model_path)